In [20]:
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
import pandas as pd
import time

In [21]:
con = sqlite3.connect("./common_stock.db")

In [22]:
cur = con.cursor()

# 종목의 업종을 저장

In [23]:
url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}"

In [24]:
test_url = url.format("005930")

In [25]:
test_url

'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930'

In [26]:
r = requests.get(test_url)

In [27]:
bs = BeautifulSoup(r.text, 'lxml')

In [28]:
stock_info = bs.find("td", {"class":"cmp-table-cell td0101"})

In [29]:
bs.find("td", class_= "cmp-table-cell td0101")

<td class="cmp-table-cell td0101">
<dl>
<dt>
<span class="name">삼성전자</span>
<a class="cEm" href="http://www.sec.co.kr" target="comPage" title="[홈페이지] www.sec.co.kr"><img align="absmiddle" hspace="3" src="/images/icon_home.gif"/></a>
<a class="cEm" title="[대표전화] 031-200-1114       
[주식담당] 02-2255-9000"><img align="absmiddle" hspace="1" src="/images/icon_tel.gif"/></a>
<b class="num">005930</b>
</dt>
<dt class="line-left">SamsungElec</dt>
<dt class="line-left">KOSPI : 전기전자</dt>
<dt class="line-left">WICS : 반도체와반도체장비</dt>
</dl>
</td>

In [30]:
stock_info

<td class="cmp-table-cell td0101">
<dl>
<dt>
<span class="name">삼성전자</span>
<a class="cEm" href="http://www.sec.co.kr" target="comPage" title="[홈페이지] www.sec.co.kr"><img align="absmiddle" hspace="3" src="/images/icon_home.gif"/></a>
<a class="cEm" title="[대표전화] 031-200-1114       
[주식담당] 02-2255-9000"><img align="absmiddle" hspace="1" src="/images/icon_tel.gif"/></a>
<b class="num">005930</b>
</dt>
<dt class="line-left">SamsungElec</dt>
<dt class="line-left">KOSPI : 전기전자</dt>
<dt class="line-left">WICS : 반도체와반도체장비</dt>
</dl>
</td>

In [31]:
stock_info.find_all(['span', 'dt'], class_ =['name', 'line-left'])

[<span class="name">삼성전자</span>,
 <dt class="line-left">SamsungElec</dt>,
 <dt class="line-left">KOSPI : 전기전자</dt>,
 <dt class="line-left">WICS : 반도체와반도체장비</dt>]

In [32]:
[x.text.split(":")[-1].strip() for x in stock_info.find_all(['span', 'dt'], class_ =['name', 'line-left'])]

['삼성전자', 'SamsungElec', '전기전자', '반도체와반도체장비']

## strip은 문자의 좌우 공백을 없애준다

# 종목코드 가져오기

In [33]:
company_data = pd.read_excel('KOSDAQCompany.xlsx')

In [44]:
company_data

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,3S,60310,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,박종익,http://www.3sref.com,서울특별시
1,APS홀딩스,54620,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
2,AP시스템,265520,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
3,AP위성,211270,통신 및 방송 장비 제조업,위성통신 단말기,2016-03-04,12월,류장수,http://www.apsi.co.kr,서울특별시
4,C&S자산관리,32040,기타 사업지원 서비스업,경비,1997-01-23,12월,구본철,http://www.shincheon.co.kr,서울특별시
...,...,...,...,...,...,...,...,...,...
1261,흥구석유,24060,기타 전문 도매업,휘발유등,1994-12-07,12월,김상우,NaN,대구광역시
1262,흥국,10240,특수 목적용 기계 제조업,굴삭기 Roller 및 형단조품,2009-05-12,12월,류명준,http://www.heungkuk.co.kr,충청남도
1263,흥국에프엔비,189980,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)",2015-08-07,12월,"박철범, 오길영",http://www.hyungkuk.com,서울특별시
1264,희림,37440,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 황의표(각자대표)",http://www.heerim.com,서울특별시


**Table 만들기**

In [35]:
query = """
        CREATE TABLE IF NOT EXISTS "stock_code_type"(
        "종목코드" TEXT NOT NULL,
        "종목이름" TEXT NOT NULL,
        "영문" TEXT NOT NULL,
        "KOSPI" TEXT,
        "WICS" TEXT,
        PRIMARY KEY('종목코드')
        )
        """

In [36]:
cur.execute(query)

In [79]:
total = []
for i in range(200):
    code = company_data['종목코드'][i]
    code = str(code)
    if(len(code) == 5):
        code = '0'+code
    print(code)
    while(True):
        try:
            r = requests.get(url.format(code))
            break
        except Exception as e:
            print(e)
            time.sleep(3)
    bs = BeautifulSoup(r.text, "lxml")
    try:
        stock_info = bs.find("td", {'class': 'cmp-table-cell td0101'})
        rt = [x.text.split(":")[-1] for x in stock_info.find_all(['span','dt'],{"class":["name","line-left"]})]
        rt.insert(0, code)
        total.append(rt)
    except Exception as e:
        print( e)

060310
054620
265520
211270
032040
'NoneType' object has no attribute 'find_all'
130960
'NoneType' object has no attribute 'find_all'
035760
051500
058820
023460
065770
083660
045890
068790
241520
037370
079190
130500
032860
900290
083450
028150
078150
036640
225430
254120
264850
276920
277410
052220
095340
099520
067290
035900
024120
021320
036670
044180
151860
046440
035600
060720
083470
122450
052900
058400
114450
036030
060370
065150
219550
160550
053290
104200
060250
030190
024940
218410
091340
019550
950110
046140
036120
099220
036540
040610
049470
184230
090730
'NoneType' object has no attribute 'find_all'
232330
'NoneType' object has no attribute 'find_all'
057500
178920
036490
052260
048550
089230
032540
048770
038340
052300
040300
051390
079940
078890
192410
024840
217730
094480
063080
039240
053950
011040
024910
089890
198440
049720
014570
098460
038530
219580
'NoneType' object has no attribute 'find_all'
227950
900280
215000
121440
014200
026910
090150
223040
'NoneType' obj

In [80]:
total_df = pd.DataFrame(total)

In [81]:
total_df.columns = ['종목코드', '종목이름', '영문', 'KOSPI', 'WICS']

In [82]:
total_df

,종목코드,종목이름,영문,KOSPI,WICS
0,060310,3S,3S Korea,"기계,장비",기계
1,054620,APS홀딩스,APS Holdings,금융,디스플레이장비및부품
2,265520,AP시스템,AP Systems,반도체,디스플레이장비및부품
3,211270,AP위성,APSI,통신장비,핸드셋
4,035760,CJ ENM,CJ ENM,방송서비스,방송과엔터테인먼트
...,...,...,...,...,...
179,045390,대아티아이,DAEATI,일반전기전자,IT서비스
180,108380,대양전기공업,DAYANG,"운송장비,부품",조선
181,048910,대원미디어,Daewon Media,도매,방송과엔터테인먼트
182,071460,위니아딤채,WINIADIMCHAE,일반전기전자,가정용기기와용품


In [83]:
total_df.to_sql("stock_code_type", con = con, if_exists = 'append', index=False)

In [85]:
df = pd.read_sql("select * from stock_code_type", con = con) 

In [86]:
df

,종목코드,종목이름,영문,KOSPI,WICS
0,060310,3S,3S Korea,"기계,장비",기계
1,054620,APS홀딩스,APS Holdings,금융,디스플레이장비및부품
2,265520,AP시스템,AP Systems,반도체,디스플레이장비및부품
3,211270,AP위성,APSI,통신장비,핸드셋
4,035760,CJ ENM,CJ ENM,방송서비스,방송과엔터테인먼트
...,...,...,...,...,...
179,045390,대아티아이,DAEATI,일반전기전자,IT서비스
180,108380,대양전기공업,DAYANG,"운송장비,부품",조선
181,048910,대원미디어,Daewon Media,도매,방송과엔터테인먼트
182,071460,위니아딤채,WINIADIMCHAE,일반전기전자,가정용기기와용품
